In [1]:
import xml.etree.ElementTree as ET
import json
import os

In [10]:
ns = {"ns1": "ns://firmenbuch.justiz.gv.at/Abfrage/v2/AuszugResponse"}

In [ ]:
with open('/Users/bencetoth/Downloads/bizrayds/node0/data1/fbp/appl_java/gesamtstand/auszuegeKurz/661613_kurz.xml') as f:
    xml_string = f.read()
    root = ET.fromstring(xml_string)
    data = parse_entry(root)
    data["partners"] = parse_partners(root)
    data["registry_entries"] = parse_registry_entries(root)
    print(data)

{'firmenbuchnummer': '661613k', 'name': 'Körpermanufaktur KG', 'legal_form': 'Kommanditgesellschaft', 'address': {'street': 'Marktstraße', 'house_number': '36', 'postal_code': '6850', 'city': 'Dornbirn', 'country': 'AUT'}, 'business_purpose': 'Betrieb einer Praxis für Physiotherapie und Chiropraktik', 'seat': 'Dornbirn', 'partners': [{'name': 'Richard Thomas Kranabetter', 'first_name': 'Richard Thomas', 'last_name': 'Kranabetter', 'birth_date': '1991-10-20', 'role': None, 'representation': None}, {'name': 'Isagani Röser', 'first_name': 'Isagani', 'last_name': 'Röser', 'birth_date': '1981-03-23', 'role': None, 'representation': None}], 'registry_entries': [{'court': 'Landesgericht Feldkirch', 'file_number': '929 048 Fr 1456/25 y', 'application_date': '2025-09-08', 'registration_date': '2025-09-09', 'type': 'Neueintragung'}, {'court': 'Landesgericht Feldkirch', 'file_number': '929 048 Fr 1553/25 s', 'application_date': '2025-09-11', 'registration_date': '2025-09-24', 'type': 'Änderung'}]

In [12]:
def get_text(element, path):
    node = element.find(path, ns)
    return node.text.strip() if node is not None and node.text else None

In [19]:
def parse_entry(root):
    data = {
        "firmenbuchnummer": root.attrib.get("{ns://firmenbuch.justiz.gv.at/Abfrage/v2/AuszugResponse}FNR", "").replace(" ", ""),
        "name": get_text(root, ".//ns1:FI_DKZ02/ns1:BEZEICHNUNG"),
        "legal_form": get_text(root, ".//ns1:FI_DKZ07/ns1:RECHTSFORM/ns1:TEXT"),
        "address": {
            "street": get_text(root, ".//ns1:FI_DKZ03/ns1:STRASSE"),
            "house_number": get_text(root, ".//ns1:FI_DKZ03/ns1:HAUSNUMMER"),
            "postal_code": get_text(root, ".//ns1:FI_DKZ03/ns1:PLZ"),
            "city": get_text(root, ".//ns1:FI_DKZ03/ns1:ORT"),
            "country": get_text(root, ".//ns1:FI_DKZ03/ns1:STAAT"),
        },
        "business_purpose": get_text(root, ".//ns1:FI_DKZ05/ns1:TEXT"),
        "seat": get_text(root, ".//ns1:FI_DKZ06/ns1:SITZ"),
        "partners": [],
        "registry_entries": [],
        "euid": get_text(root, ".//ns1:EUID/ns1:EUID"),
        "reference_date": root.attrib.get("{ns://firmenbuch.justiz.gv.at/Abfrage/v2/AuszugResponse}STICHTAG"),
        "query_timestamp": root.attrib.get("{ns://firmenbuch.justiz.gv.at/Abfrage/v2/AuszugResponse}ABFRAGEZEITPUNKT"),
    }
    return data

In [21]:
def parse_partners(root):
    partners = []
    for per in root.findall(".//ns1:PER", ns):
        pnr = per.attrib.get("{ns://firmenbuch.justiz.gv.at/Abfrage/v2/AuszugResponse}PNR", "").strip()
        first = get_text(per, ".//ns1:PE_DKZ02/ns1:VORNAME")
        last = get_text(per, ".//ns1:PE_DKZ02/ns1:NACHNAME")
        name = get_text(per, ".//ns1:PE_DKZ02/ns1:NAME_FORMATIERT")
        birth = get_text(per, ".//ns1:PE_DKZ02/ns1:GEBURTSDATUM")
        
        # Find role & representation from FUN with matching PNR
        fun = root.find(f".//ns1:FUN[@ns1:PNR='{pnr}']", ns)
        role = fun.attrib.get("{ns://firmenbuch.justiz.gv.at/Abfrage/v2/AuszugResponse}FKENTEXT") if fun is not None else None
        representation = get_text(fun, ".//ns1:FU_DKZ10/ns1:VART/ns1:TEXT") if fun is not None else None

        partners.append({
            "name": name,
            "first_name": first,
            "last_name": last,
            "birth_date": f"{birth[:4]}-{birth[4:6]}-{birth[6:]}" if birth else None,
            "role": role,
            "representation": representation
        })
    return partners

In [23]:
def parse_registry_entries(root):
    registry_entries = []   
    for vollz in root.findall(".//ns1:VOLLZ", ns):
        entry = {
            "court": get_text(vollz, "ns1:HG/ns1:TEXT"),
            "file_number": get_text(vollz, "ns1:AZ"),
            "application_date": get_text(vollz, "ns1:EINGELANGTAM"),
            "registration_date": get_text(vollz, "ns1:VOLLZUGSDATUM"),
            "type": "Änderung" if "Änderung" in (get_text(vollz, "ns1:ANTRAGSTEXT") or "") else "Neueintragung"
        }
        registry_entries.append(entry)
    return registry_entries